In [ ]:
#importing the libraries
import math
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:

data = pd.read_csv('/content/AAPL.csv')



In [ ]:
data

In [ ]:
data.isnull().sum()

In [ ]:
data.shape

**Let us graphically represent the closing prices of the stock.**

In [ ]:
plt.figure(figsize=(16,8))
plt.title('Closing Price of the Stock Historically')
plt.plot(data['Close'])
plt.xlabel('Year', fontsize=20)
plt.ylabel('Closing Price Historically ($)', fontsize=20)
plt.show()

**LSTM Algorithm**

We need to create a seperate dataframe with the "close" column

In [ ]:
data = data.filter(['Close'])
dataset = data.values

#Find out the number of rows that are present in this dataset in order to train our model.
training_data_len = math.ceil(len(dataset)* .8)
training_data_len

Now, we need to scale the data

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)
scaled_data


Create the training data for the model

In [ ]:
train_data = scaled_data[0:training_data_len , :]
x_train = []
y_train = []

for j in range(60, len(train_data)):
    x_train.append(train_data[j-60:j,0])
    y_train.append(train_data[j,0])
    if j<=60:
        print(x_train)
        print(y_train)
        print()

In [ ]:
x_train, y_train = np.array(x_train), np.array(y_train)

In [ ]:
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_train.shape

**Building the LSTM Model**

In [ ]:
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape = (x_train.shape[1], 1)))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error')

**Training the Model**

In [ ]:
model.fit(x_train, y_train, batch_size=1, epochs=1)

In [ ]:
test_data = scaled_data[training_data_len - 60: , :]
x_test = []
y_test = dataset[training_data_len:, :]

for j in range(60, len(test_data)):
    x_test.append(test_data[j-60:j, 0])

In [ ]:
x_test = np.array(x_test)

In [ ]:
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

In [ ]:
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)

Finding out the Root Mean Squared Error

In [ ]:
rmse = np.sqrt( np.mean( predictions - y_test)**2)
rmse

**Plot the graph**

In [ ]:
train = data[:training_data_len]
val = data[training_data_len:]
val['Predictions'] = predictions

plt.figure(figsize=(16,8))
plt.title('LSTM Model Data')
plt.xlabel('Date', fontsize=16)
plt.ylabel('Close Price', fontsize=16)
plt.plot(train['Close'])
plt.plot(val[['Close', 'Predictions']])
plt.legend(['Trained Dataset', 'Actual Value', 'Predictions'])
plt.show()

In [ ]:
val.head(10)

In [ ]:
new_data = pd.read_csv('/content/AAPL.csv')
new_data = data.filter(['Close'])
last_60_days = new_data[-60:].values
last_60_scaled = scaler.transform(last_60_days)
X_test = []
X_test.append(last_60_scaled)
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
predicted_price = model.predict(X_test)
predicted_price = scaler.inverse_transform(predicted_price)
print('The predicted price of the final value of the dataset', predicted_price)

In [ ]:
new_data.tail(1)

In [ ]:
#check predicted values
predictions = model.predict(x_test) 
#Undo scaling
predictions = scaler.inverse_transform(predictions)

#Calculate RMSE score
rmse=np.sqrt(np.mean(((predictions- y_test)**2)))
rmse

In [ ]:
neww_data = pd.read_csv('/content/AAPL.csv')

In [ ]:
val.describe()

In [ ]:
x = val.Close.mean()
y = val.Predictions.mean()
Accuracy = x/y*100
print("The accuracy of the model is " , Accuracy)

**Regression Model**

In [ ]:
from sklearn import metrics
%matplotlib inline
import matplotlib.pyplot as plt
import math

In [ ]:
data = pd.read_csv('/content/AAPL.csv')
data

In [ ]:
plt.scatter(data.Date, data.Close,  color='blue')
plt.xlabel("Time")
plt.ylabel("Close")
plt.show()

In [ ]:
dataset.shape

In [ ]:
data['Date'] = pd.to_datetime(data.Date)

In [ ]:
data.head()

In [ ]:
data.describe()

In [ ]:
print(len(data))

In [ ]:
data['Close'].plot(figsize=(16,8))

In [ ]:
x1 = data[['Open', 'High','Low', 'Volume']]
y1 = data['Close']

In [ ]:
from sklearn.model_selection import train_test_split
x1_train, x1_test, y1_train, y1_test = train_test_split(x1, y1, random_state = 0)

In [ ]:
x1_train.shape

In [ ]:
x1_test.shape

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import confusion_matrix, accuracy_score
regression = LinearRegression()

In [ ]:
regression.fit(x1_train, y1_train)

In [ ]:
print(regression.coef_)

In [ ]:
print(regression.intercept_)

In [ ]:
predicted=regression.predict(x1_test)



In [ ]:
print(x1_test)

In [ ]:
predicted.shape

In [ ]:
dframe = pd.DataFrame(y1_test,predicted)
dfr=pd.DataFrame({'Actual_Price':y1_test, 'Predicted_Price':predicted})
print(dfr)

In [ ]:
dfr.head(10)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [ ]:
regression.score(x1_test, y1_test)

In [ ]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y1_test, predicted))
print('Mean Squared Error:', metrics.mean_squared_error(y1_test, predicted))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y1_test, predicted)))

In [ ]:
dfr.describe()

In [ ]:
x2 = dfr.Actual_Price.mean()
y2 = dfr.Predicted_Price.mean()
Accuracy1 = x2/y2*100
print("The accuracy of the model is " , Accuracy1)

In [ ]:
plt.scatter(dfr.Actual_Price, dfr.Predicted_Price,  color='red')
plt.xlabel("Actual Price")
plt.ylabel("Predicted Price")

plt.show()